## CM top 1024, sketch 1024*4
- Find:209,TP:0,FP:209
- avg error:432.427734375

## space saving top1024
- Find:180,TP:29,FP:151
- avg error:1.62109375

## top 1024, sketch 128*4
- width*1
    - Find:214,TP:56,FP:158
    - avg error:1.333984375
- width*1.1
    - Find:244,TP:60,FP:184
    - avg error:1.4765625
- width*1.2   
    - Find:211,TP:83,FP:128
    - avg error:0.716796875 
- width*1.5
    - Find:181,TP:94,FP:87
    - avg error:0.6328125
## top 512, sketch 128*4 
- width*1
    - Find:132,TP:25,FP:107
    - avg error:1.86328125
- width*1.1
    - Find:132,TP:35,FP:97
    - avg error:1.29296875
- width*1.2   
    - Find:106,TP:23,FP:83
    - avg error:1.21875   
- width*1.5
    - Find:117,TP:18,FP:99
    - avg error:3.1875

## About Kosarak
- Distinct element: 606770
- Number of incomming element: 990002
- Top-10 frequent element and count: 

| element | frquent |
| :----- | :----- |
| 3 | 36133 |
| 6 | 30225 |
| 6 3 | 21139 |
| 11 6 | 12769 |
| 1 | 9766 |
| 11 6 3 | 9083 |
| 1 3 | 6002 |
| 11 | 5374 |
| 1 6 | 5046 |
| 4 | 4290 |


In [7]:
# Ground Truth
import pandas as pd
import os

filename='kosarak.dat'
filepath="..\\dataset\\"
src_data=os.path.join(filepath,filename)

num_line=0
item_count=100000
itemdict={}
with open(src_data,'r') as file:
    while item_count:
        line=file.readline().strip('\n')
        if not line:
            break
        else:
            item_count-=1
            num_line+=1
            if line not in itemdict:
                itemdict[line]=1
            else:
                itemdict[line]+=1

df=pd.DataFrame(list(itemdict.items()),columns=['Element', 'Count'])
df=df.reset_index(drop=True)
df=df.sort_values('Count',ascending=False)

print("Distinct element: {}".format(len(itemdict)))
print("Number of incomming element: {}".format(num_line))
print(df.head(20))

df.to_csv("..\\result\\kosarak\\kosarak_100k_ground_truth.csv",index=False)


Distinct element: 69007
Number of incomming element: 100000
     Element  Count
17         3   3656
111        6   3038
31       6 3   2135
94      11 6   1325
1          1   1010
85    11 6 3    894
271      1 3    593
37       1 6    514
253       11    486
279        4    442
86      11 3    405
466    1 6 3    371
241       55    339
233      4 3    320
64      55 3    255
195   11 1 6    235
1525     4 6    217
262        2    197
116      294    172
89      6 55    170


## Count-min sketch
- Top-1024+ 4*1024 CMS
    - Total memory 9432 bytes :Top-1024 with size 9328 bytes+ CMS with size 104 bytes.
    - Execution time:319.18891406059265 seconds
    - built-in index search is fastest

In [3]:
# Count-Min sketch with built-in index search
# Top-1024+ 4*1024 CMS
# Total memory 9432 bytes :Top-1024 with size 9328 bytes+ CMS with size 104 bytes.
# Execution time:319.18891406059265 seconds.
# Find:339,TP:0,FP:339

import sys
import os
import time
from probables import (CountMinSketch)
import pandas as pd

def find(e,Topk):
    try:
        index = [ele for ele,i in Topk].index(e)
        return index
    except:
        index=-99
    return index

start=time.time()
filename='kosarak.dat'
filepath="..\\dataset\\"
src_data=os.path.join(filepath,filename)
topk=[]
size=1024
# item_count=10000
cms = CountMinSketch(width=1024, depth=4)

if os.path.exists(src_data):
    with open(src_data,'r') as file:
        while True:
            line=file.readline().strip('\n')
            if not line:
                print('EOF')
                break
            else:
                #item_count-=1
                # print("read {}th element: {}".format(item_count,element))
                cms.add(line)
                count=cms.check(line)
                if len(topk)==0:
                    topk.append([line,count])
                else:
                    index=find(line,topk)
                    if index<0:
                        #  element not in topk
                        if len(topk)<size:
                            topk.append([line,count])
                        else:
                            topk[-1][0]=line
                            topk[-1][1]=count
                    else:
                        topk[index][1]=count
                topk=sorted(topk,key = lambda topk:topk[1],reverse=True)
                    
    end=time.time()
    print(topk[:20],len(topk))
    print("Total memory {3} bytes :Top-{0} with size {1} bytes+ CMS with size {2} bytes.".format(len(topk),sys.getsizeof(topk),sys.getsizeof(cms),sys.getsizeof(cms)+sys.getsizeof(topk)))
    print("Execution time:{} seconds.".format(str(end-start)))
else:
    print("file doesn't exist")
#　conver Top into df    
templi=[]
for i in topk:
    templi.append([i[0],i[1]])

df=pd.DataFrame(templi,columns=['ID', 'Count'])
df.to_csv("..\\result\\kosarak\\CM_kosarak.csv",index=False)
df.head(50)


EOF
[['3', 36826], ['6', 30906], ['6 3', 21876], ['11 6', 13465], ['1', 10512], ['11 6 3', 9726], ['1 3', 6663], ['11', 6049], ['1 6', 5695], ['4', 4969], ['11 3', 4564], ['55', 4269], ['1 6 3', 4156], ['4 3', 3666], ['55 3', 3050], ['11 1 6', 3005], ['4 6', 2992], ['4 6 3', 2441], ['11 1 6 3', 2437], ['6 55', 2430]] 1024
Total memory 9432 bytes :Top-1024 with size 9328 bytes+ CMS with size 104 bytes.
Execution time:421.7094202041626 seconds.


,ID,Count
0,3,36826
1,6,30906
2,6 3,21876
3,11 6,13465
4,1,10512
5,11 6 3,9726
6,1 3,6663
7,11,6049
8,1 6,5695
9,4,4969


In [4]:
# Count-Min sketch with sequential search replace last
# Top-1024+ 4*1024 CMS
# Total memory 9432 bytes :Top-1024 with size 9328 bytes+ CMS with size 104 bytes.
# Execution time:355.103120803833 seconds.
# Find:339,TP:0,FP:339

import sys
import os
import time
from probables import (CountMinSketch)
import pandas as pd

start=time.time()
filename='kosarak.dat'
filepath="..\\dataset\\"
src_data=os.path.join(filepath,filename)

topk=[]
size=1024
item_count=0
cms = CountMinSketch(width=1024, depth=4)

if os.path.exists(src_data):
    with open(src_data,'r') as file:
        while True:
            line=file.readline().strip('\n')
            if not line:
                print('EOF')
                break
            else:
                # item_count+=1
                # print("read {}th element: {}".format(item_count,element))
                cms.add(line)
                count=cms.check(line)
                if len(topk)==0:
                    topk.append([line,count])
                else:
                    find=False
                    for i in range(len(topk)):
                        if line==topk[i][0]:
                            topk[i][1]=count
                            find=True
                            break
                    if find==False:
                        if len(topk)<size:
                            topk.append([line,count])
                        else:
                            topk[-1][0]=line
                            topk[-1][1]=count
                topk=sorted(topk,key = lambda topk:topk[1],reverse=True)
                    
    end=time.time()
    print(topk[:50],len(topk))
    print("Total memory {3} bytes :Top-{0} with size {1} bytes+ CMS with size {2} bytes.".format(len(topk),sys.getsizeof(topk),sys.getsizeof(cms),sys.getsizeof(cms)+sys.getsizeof(topk)))
    print("Execution time:{} seconds.".format(str(end-start)))
else:
    print("file doesn't exist")

#　conver Top into df    
templi=[]
for i in topk:
    templi.append([i[0],i[1]])

df=pd.DataFrame(templi,columns=['ID', 'Count'])
df.to_csv("..\\result\\kosarak\\CM_kosarak_replace_last.csv",index=False)
df.head(20)


EOF
[['3', 36826], ['6', 30906], ['6 3', 21876], ['11 6', 13465], ['1', 10512], ['11 6 3', 9726], ['1 3', 6663], ['11', 6049], ['1 6', 5695], ['4', 4969], ['11 3', 4564], ['55', 4269], ['1 6 3', 4156], ['4 3', 3666], ['55 3', 3050], ['11 1 6', 3005], ['4 6', 2992], ['4 6 3', 2441], ['11 1 6 3', 2437], ['6 55', 2430], ['2', 2310], ['294', 2276], ['6 55 3', 2004], ['490', 1981], ['2 3', 1946], ['27', 1822], ['7', 1770], ['294 3', 1759], ['218', 1717], ['11 4 6', 1594], ['490 3', 1578], ['11 1', 1554], ['6 2', 1540], ['6 294', 1534], ['64', 1521], ['73', 1495], ['11 4 6 3', 1488], ['91', 1488], ['4 1', 1438], ['218 3', 1434], ['11 6 55', 1412], ['11 6 55 3', 1409], ['6 2 3', 1389], ['3 7', 1373], ['490 6', 1365], ['6 7', 1365], ['6 294 3', 1354], ['27 3', 1349], ['218 6', 1324], ['77', 1311]] 1024
Total memory 9432 bytes :Top-1024 with size 9328 bytes+ CMS with size 104 bytes.
Execution time:506.42608761787415 seconds.


,ID,Count
0,3,36826
1,6,30906
2,6 3,21876
3,11 6,13465
4,1,10512
5,11 6 3,9726
6,1 3,6663
7,11,6049
8,1 6,5695
9,4,4969


In [9]:
# Result compare
# ARE: (1/distinct element)*(abs(f-f*)/f)

import pandas as pd
import os
path='..\\result\\kosarak'
groundtruth='kosarak_ground_truth.csv'
final='CM_kosarak.csv'

grtruth=pd.read_csv(os.path.join(path,groundtruth))
My_result1=pd.read_csv(os.path.join(path,final))

gli = grtruth.values.tolist()
li1= My_result1.values.tolist()

tp=0
fp=0
find=0
err=[]
error=0
for item in li1:
    for element in gli[:1024]:
        if item[0]==element[0]:
            # print("{},{} vs. {},{}".format(item[0],item[1],element[0],element[1]))
            find+=1
            if item[1]==element[1]:
                tp+=1
            else:
                fp+=1
                error+=abs(item[1]-element[1])/item[1]
print("Find:{},TP:{},FP:{}".format(find,tp,fp))
print("ARE:{}".format(error/606770))

Find:339,TP:0,FP:339
ARE:0.00040978478569307744


In [10]:
## ARE

import pandas as pd
import os
path='..\\result\\kosarak'
groundtruth='kosarak_ground_truth.csv'
final='CM_kosarak.csv'

grtruth=pd.read_csv(os.path.join(path,groundtruth))
My_result1=pd.read_csv(os.path.join(path,final))

gli = grtruth.values.tolist()
li= My_result1.values.tolist()

err=0
for item in li:
    index=[j[0] for j in gli[:]].index(item[0])
    # print("index of {}:{}".format(item,))
    err+=abs(item[1]-gli[index][1])/item[1]
print(err/606770)


0.0015354806018180271


## Space-Saving


In [1]:
# Space-Saving 1024 stream summary with built-in index search
# Total memory 9328 bytes :Top-1024 with size 9328 bytes.
# Execution time:254.7294762134552 seconds.
# Find:158,TP:26,FP:132

import sys
import time

def find(e,Topk):
    try:
        index = [ele for ele,i in Topk].index(e)
        return index
    except:
        index=-99
    return index


start=time.time()
filename='kosarak.dat'
filepath="..\\dataset\\"
src_data=os.path.join(filepath,filename)

size=1024
itemlist=[]
item_count=10000
with open(src_data,'r') as file:
    while item_count:
        line=file.readline().strip('\n')
        if not line:
            break
        else:
            item_count-=1
            if len(itemlist)==0:
                itemlist.append([line,1])
            else:
                index=find(line,itemlist)
                if index<0:
                    if len(itemlist)<size:
                        itemlist.append([line,1])
                            # add new element into itemlist
                    else:
                        # replace last element with count 
                        itemlist[-1][0]=line
                        itemlist[-1][1]+=1
                else:
                    itemlist[index][1]+=1
            itemlist=sorted(itemlist,key = lambda itemlist: itemlist[1],reverse=True)
        #print("itemlist:{}\nlen(itemlist):{}\n".format(itemlist,len(itemlist)))
end=time.time()

print(itemlist[:20],len(itemlist))
print("Total memory {0} bytes :Top-{1} with size {0} bytes.".format(sys.getsizeof(itemlist),size))
print("Execution time:{} seconds.".format(str(end-start)))



#　conver Top into df    
templi=[]
for i in itemlist:
    templi.append([i[0],i[1]])

df=pd.DataFrame(templi,columns=['ID', 'Count'])
df.to_csv("..\\result\\kosarak\\SS_kosarak_index_search.csv",index=False)
df.head(20)




NameError: name 'os' is not defined

In [1]:
# Space-Saving 1024 stream summary with class+index search
# Total memory 9024 bytes :Top-1024 with size 9024 bytes.
# Execution time:251.02409100532532 seconds.
# Find:158,TP:26,FP:132

import sys
import os
import time
import operator
import pandas as pd

class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'ID: {}, count: {}'.format(self.ID,self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self):
        super().__init__()
        self.distinct = hyperloglog.HyperLogLog(0.01)
    def __str__(self):
        return 'total count: {}, distinct element: {}'.format(self.count,len(self.distinct))
    def __repr__(self):
        return '[count: {}, distinct: {}]'.format(self.count,len(self.distinct))

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return 'ID: {}, count: {}'.format(self.ID,self.count)
    def __repr__(self):
        return "'{}', count: {}".format(self.ID,self.count)

def find(e,element_list):
    try:
        index = [ele.ID for ele in element_list].index(e.ID)
    except:
        index=-99
    return index  
    
    
start=time.time()

filename='kosarak.dat'
filepath="..\\dataset\\"
src_data=os.path.join(filepath,filename)

size=1024
Top=[]
#item_count=100000

with open(src_data,'r') as file:
    while True:
        element=file.readline().strip('\n')
        if not element:
            print('EOF')
            break
        else:
            item=Tail(element,1)
            #item_count-=1
            # print("read {}th element: {}".format(item_count,element))
            if len(Top)==0:
                Top.append(item)
            else:
                index=find(item,Top)
                if index<0:
                    if len(Top)<size:
                        Top.append(item)
                    else:
                        # replace last element with count 
                        Top[-1].ID=item.ID
                        Top[-1].count+=1
                else:
                    Top[index].count+=1
            Top.sort(key=operator.attrgetter('count'),reverse=True)

end=time.time()
print(Top[:20],len(Top))
print("Total memory {0} bytes :Top-{1} with size {0} bytes.".format(sys.getsizeof(Top),size))
print("Execution time:{} seconds.".format(str(end-start)))


#　conver Top into df    
templi=[]
for i in Top:
    templi.append([i.ID,i.count])

df=pd.DataFrame(templi,columns=['ID', 'Count'])
df.to_csv("..\\result\\kosarak\\SS_class_kosarak.csv",index=False)
df.head(20)



EOF
['3', count: 36133, '6', count: 30225, '6 3', count: 21139, '11 6', count: 12769, '1', count: 9766, '11 6 3', count: 9083, '1 3', count: 6002, '11', count: 5374, '1 6', count: 5046, '4', count: 4290, '11 3', count: 3756, '55', count: 3561, '1 6 3', count: 3510, '4 3', count: 3000, '55 3', count: 2377, '11 1 6', count: 2375, '4 6', count: 2341, '4 6 3', count: 1758, '11 1 6 3', count: 1753, '6 55', count: 1749] 1024
Total memory 9024 bytes :Top-1024 with size 9024 bytes.
Execution time:306.57678055763245 seconds.


,ID,Count
0,3,36133
1,6,30225
2,6 3,21139
3,11 6,12769
4,1,9766
5,11 6 3,9083
6,1 3,6002
7,11,5374
8,1 6,5046
9,4,4290


In [70]:
# Space-Saving 1024 stream summary with sequential search
# Total memory 9328 bytes :Top-1024 with size 9328 bytes.
# Execution time:291.83072996139526 seconds.
# Find:158,TP:26,FP:132

import sys
import time

start=time.time()

filename='kosarak.dat'
filepath="..\\dataset\\"
src_data=os.path.join(filepath,filename)

size=1024
itemlist=[]
item_count=10000
with open(src_data,'r') as file:
    while True:
        line=file.readline().strip('\n')
        if not line:
            break
        else:
            # item_count-=1
            if len(itemlist)==0:
                itemlist.append([line,1])
            else:
                for i in range(len(itemlist)):
                    if line==itemlist[i][0]:
                        itemlist[i][1]+=1
                        found=True
                        #print("found in itemlist[{}]".format(i))
                        break
                    else:
                        found=False
                if found:
                    pass
                else:
                    if len(itemlist)<size:
                        itemlist.append([line,1])
                    else:
                        # len(itemset)=size
                        itemlist[-1][0]=line
                        itemlist[-1][1]+=1
            itemlist=sorted(itemlist,key = lambda itemlist: itemlist[1],reverse=True)
        #print("itemlist:{}\nlen(itemlist):{}\n".format(itemlist,len(itemlist)))
end=time.time()

print(itemlist[:20],len(itemlist))
print("Total memory {0} bytes :Top-{1} with size {0} bytes.".format(sys.getsizeof(itemlist),size))
print("Execution time:{} seconds.".format(str(end-start)))

#　conver Top into df    
templi=[]
for i in itemlist:
    templi.append([i[0],i[1]])

df=pd.DataFrame(templi,columns=['ID', 'Count'])
df.to_csv("..\\result\\kosarak\\SS_kosarak_o.csv",index=False)
df.head(20)

[['3', 36133], ['6', 30225], ['6 3', 21139], ['11 6', 12769], ['1', 9766], ['11 6 3', 9083], ['1 3', 6002], ['11', 5374], ['1 6', 5046], ['4', 4290], ['11 3', 3756], ['55', 3561], ['1 6 3', 3510], ['4 3', 3000], ['55 3', 2377], ['11 1 6', 2375], ['4 6', 2341], ['4 6 3', 1758], ['11 1 6 3', 1753], ['6 55', 1749]] 1024
Total memory 9328 bytes :Top-1024 with size 9328 bytes.
Execution time:291.83072996139526 seconds.


,ID,Count
0,3,36133
1,6,30225
2,6 3,21139
3,11 6,12769
4,1,9766
5,11 6 3,9083
6,1 3,6002
7,11,5374
8,1 6,5046
9,4,4290


In [10]:
# SS Result compare
import pandas as pd
import os
path='..\\result\\kosarak'
groundtruth='kosarak_100k_ground_truth.csv'
built_in='SS_kosarak_index_search.csv'
origin='SS_kosarak_o.csv'
final='SS_class_kosarak_100k_count.csv'

grtruth=pd.read_csv(os.path.join(path,groundtruth))
My_result1=pd.read_csv(os.path.join(path,final))

gli = grtruth.values.tolist()
li1= My_result1.values.tolist()

tp=0
fp=0
find=0
err=[]
error=0
for item in li1:
    for element in gli[:1024]:
        if item[0]==element[0]:
            print("{},{} vs. {},{}".format(item[0],item[1],element[0],element[1]))
            find+=1
            if item[1]==element[1]:
                tp+=1
            else:
                fp+=1
                error+=abs(item[1]-element[1])
print("Find:{},TP:{},FP:{}".format(find,tp,fp))
print("avg error:{}".format(error/512))

3,3656 vs. 3,3656
6,3038 vs. 6,3038
6 3,2135 vs. 6 3,2135
11 6,1325 vs. 11 6,1325
1,1010 vs. 1,1010
11 6 3,894 vs. 11 6 3,894
1 3,593 vs. 1 3,593
1 6,514 vs. 1 6,514
11,486 vs. 11,486
4,442 vs. 4,442
11 3,405 vs. 11 3,405
1 6 3,371 vs. 1 6 3,371
55,339 vs. 55,339
4 3,320 vs. 4 3,320
55 3,255 vs. 55 3,255
11 1 6,235 vs. 11 1 6,235
4 6,218 vs. 4 6,217
2,197 vs. 2,197
294,172 vs. 294,172
6 55,170 vs. 6 55,170
4 6 3,167 vs. 4 6 3,167
6 55 3,158 vs. 6 55 3,157
11 1 6 3,157 vs. 11 1 6 3,157
294 3,141 vs. 294 3,140
490,126 vs. 490,125
7,118 vs. 7,118
218,115 vs. 218,114
11 4 6,109 vs. 11 4 6,109
27,105 vs. 27,104
2 3,101 vs. 2 3,101
11 1,100 vs. 11 1,100
6 2,96 vs. 6 2,96
490 3,91 vs. 490 3,84
6 294,91 vs. 6 294,88
11 6 55 3,90 vs. 11 6 55 3,80
6 294 3,89 vs. 6 294 3,74
64,87 vs. 64,79
11 4 6 3,86 vs. 11 4 6 3,82
3 73,86 vs. 3 73,66
218 6,85 vs. 218 6,67
246,85 vs. 246,43
18,85 vs. 18,42
218 3,85 vs. 218 3,78
6 278,85 vs. 6 278,25
316,84 vs. 316,78
3 64,84 vs. 3 64,48
1 55,84 vs. 1 55,60
3 7,

## My algo

In [81]:
# Top-512+ 4*128 SK with sequential search
# Total memory 4272 bytes :Top-512 with size 4272 bytes.
# Execution time:238.59611749649048 seconds.
# Find:165,TP:36,FP:129

import sys
import time
import spookyhash
import mmh3
import hyperloglog
import operator
import unittest
from numpy import random

# ==========================data structure==========================
class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'count: {}'.format(self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self,count=1):
        super().__init__(count)
        self.distinct = hyperloglog.HyperLogLog(0.01)
        self.maxID=''
    def __str__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))
    def __repr__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
    def __repr__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
# ==========================data structure==========================

# ========================== Update Sketch==========================
def UpdateSk(element,width,depth):
    #　width+=1
    e_max=get_emax()
    # print("first read e_max in UpdateSK, id(e_max):{}".format(id(e_max)))
    hash1=spookyhash.hash32(bytes((element.ID),encoding='utf-8'))
        # input of spooky hash: binary, encoding is parameter of bytes()
        # output of spooky hash: unsigned- 32 bit int
    hash2=mmh3.hash(str(hash1), signed=False)
        # input of mmh3 is str, output unsigned- 32 bit int
    row=hash1 % depth
        # index of row in SK[row]
    ID=hash2 % ((width*3)//2)
        # hash-value(ID) of e
        # range: 2^k *3/2= 3*(2^k-1)
    match=False
        # match= True if e is in Sk[row]
    index=0
    item=Tail(ID,element.count)
    
    # print('UpdateSk:{}->ID {}'.format(element,ID))
    # print("hash1: {}".format(hash1))
    # print("hash2: {}".format(hash2))
    # print("match row: {}".format(row))
    # print("SK[{}]:{}".format(row,Sk[row]))
    # print("len(Sk[{}]:{})".format(row,len(Sk[row])))
    
    Sk[row][0].add_count(item.count)
        # total count+=count
    if len(Sk[row])==1:
        # Head only, append e directly
        Sk[row].append(item)
        Sk[row][0].maxID=element.ID
        match=True
        index=1
            # e is in Sk[row][1]
        # print('Sk[{}] when len(Sk[{}])==1:\n\t{}'.format(row,row,Sk[row]))
        # print("After append:\n{}".format(Sk[row]))
        # print("length of Sk[row]: {}".format(len(Sk[row])))
    elif len(Sk[row])<width:
        put_in=random.choice([0,1],size=1,p=[len(Sk[row])/width,1-len(Sk[row])/width])[0]
            # send into SK[row] with prob. 1-len(SK[row])/width
        for i in range(1,len(Sk[row])):
            # checks whether e is in SK[row]
            if ID == Sk[row][i].ID:
                match=True
                Sk[row][i].add_count(item.count)
                index=i
                '''
                print('{} matches in Sk[{}][{}]:{}'.format(item.ID,row,index,Sk[row][index].ID))
                print('Sk[{}]:\n\t{}'.format(row,Sk[row]))
                '''

                break
            else:
                match=False
        if not match:
            if put_in==1:
                # send e into SK[row]
                # print('put_in={}, send e into SK[{}]'.format(put_in,row))
                Sk[row].append(item)
                match=True
                index=len(Sk[row])-1
                # print('Sk[{}]:\n\t{}\n'.format(row,Sk[row]))
            else:
                Sk[row][0].distinct.add(ID)
                ''' 
                print('put_in={}, send e into other,update distinct'.format(put_in,len(Sk[row][0].distinct)))
                print('Sk[{}]:\n\t {}'.format(row,Sk[row]))
                '''

    elif len(Sk[row])==width:
        # len(Sk[row])=width, put e into other
        Sk[row][0].distinct.add(ID)

    if match:
        # print("e_max in UpdateSK line 114, id(e_max):{}".format(id(e_max)))
        if Sk[row][index].count>e_max.count:
            e_max.ID=element.ID
            e_max.count=Sk[row][index].count
        # print('line 115 e_max: e_max={}'.format(e_max))
            
    else:
        # avg count of other > e_max.count
        avg=(Sk[row][0].count-sum(Sk[row][i].count for i in range(len(Sk[row]))))/len(Sk[row][0].distinct)
        if avg >e_max.count:
            e_max=Tail(element.ID,avg)
        # print('line 122 e_max: e_max={}'.format(e_max))
    # print('line 123 e_max:{}'.format(get_emax()))
    # print('')
# ========================== Update Sketch==========================

# ========================== BringBack=========================
def BringBack(e_min,e_max):
    # print("Top before Bringback:\n\t{}".format(Top))
    e_max=get_emax()
    # print('e_max at first Bringback:{},id(e_max):{}'.format(e_max,id(e_max)))
    temp=Tail(e_min.ID,e_min.count)
    Top[-1].ID=e_max.ID
    Top[-1].count=e_max.count
    Top.sort(key=operator.attrgetter('count'),reverse=True)
    # print('Top after BringBack:\n\t{}'.format(Top))
    DeleteSk(e_max)
        # e_max in Sk[row]
    e_max.ID=''
    e_max.count=0
    # print("e_max after delete:{},id(e_max):{}".format(e_max,id(e_max)))
    UpdateSk(temp,width,depth)
    
    # print("Sk[] after Update {}:\n\t{}".format(e_min,Sk))

# ========================== BringBack=========================
# ==========================DeleteSk=========================
def DeleteSk(element):
    # 刪除e_max，找新的e_max
    # print('Delete start======================================')
    hash1=spookyhash.hash32(bytes((element.ID),encoding='utf-8'))
    hash2=mmh3.hash(str(hash1), signed=False)
    ID=hash2 % ((width*3)//2)
    row=hash1 % depth
    index=0
    for i in range(1,len(Sk[row])):
        if Sk[row][i].ID==ID:
            # print("found {} at SK[{}][{}], ID={}".format(element.ID,row,i,Sk[row][i].ID))
            index=i
            # print("pop element:{}".format(Sk[row][index]))
            Sk[row].pop(index)
            break
    # print('Delete Sk for find max of Sk[i] len(Sk):{}'.format(len(Sk)))
    Sk[row][0].count-=element.count
    '''
    print("index:{}".format(index))
    print('After pop:')
    print("\tSk[{}]={}".format(row,Sk[row]))
    print("Top:\n\t{}".format(Top))
    print('Delete Sketch over======================================')
    '''
# ==========================DeleteSk=========================
# ========================== Util Functions==========================
def find(e,Topk):
    try:
        index = [ele for ele,i in Topk].index(e)
        return index
    except:
        index=-99
    return index

def get_emax():
    return e_max
# ========================== Util Functions==========================

start=time.time()
filename='kosarak.dat'
filepath="..\\dataset\\"
src_data=os.path.join(filepath,filename)
depth=4
width=128
size=512

Top=[]
Sk=[[Head(0)] for j in range(depth)]
item_count=1000

e_max=Tail('',0)


with open(src_data,'r') as file:
    while True:
        element=file.readline().strip('\n')
        # print("read element: {}".format(element))
        if not element:
            break
        else:
            # Update_Top(Tail(element,1))
            # item_count-=1
            item=Tail(element,1)
            if len(Top)==0:
                Top.append(item)
            else:
                found=False
                for i in range(len(Top)):
                    if Top[i].ID==element:
                        found=True
                        Top[i].add_count(1)
                        break
                if not found:
                    if len(Top)<size:
                        Top.append(item)
                    else:
                        # print("e_max in main, id(e_max):{}".format(id(e_max)))
                        UpdateSk(item,width,depth)
            Top.sort(key=operator.attrgetter('count'),reverse=True)
            # print("Top:{}".format(Top))
            if e_max.count>Top[-1].count:
                BringBack(Top[-1],e_max)
                # bring back
                # 要去掉SK[row]中的e_max.ID及count的node
                # print('Top after BringBack: \n\t{}'.format(Top))

end=time.time()
print(Top[:20],len(Top))
print("Total memory {1} bytes :Top-{3}: {0} bytes, Sketch: {2} bytes."
      .format(sys.getsizeof(Top),sys.getsizeof(Top)+sys.getsizeof(Sk),sys.getsizeof(Sk),size))
print("Execution time:{} seconds.".format(str(end-start)))

#　conver Top into df    
templi=[]
for i in Top:
    templi.append([i.ID,i.count])

df=pd.DataFrame(templi,columns=['ID', 'Count'])
df.to_csv("..\\result\\kosarak\\My_kosarak_o.csv",index=False)
df.head(20)


[[ID: 3, count: 36133], [ID: 6, count: 30225], [ID: 6 3, count: 21139], [ID: 11 6, count: 12769], [ID: 1, count: 9766], [ID: 11 6 3, count: 9083], [ID: 1 3, count: 6002], [ID: 11, count: 5374], [ID: 1 6, count: 5046], [ID: 4, count: 4290], [ID: 11 3, count: 3756], [ID: 55, count: 3561], [ID: 1 6 3, count: 3510], [ID: 4 3, count: 3000], [ID: 55 3, count: 2377], [ID: 11 1 6, count: 2375], [ID: 4 6, count: 2341], [ID: 4 6 3, count: 1754], [ID: 11 1 6 3, count: 1753], [ID: 6 55, count: 1750]] 512
Total memory 4272 bytes :Top-512 with size 4272 bytes.
Execution time:238.59611749649048 seconds.


,ID,Count
0,3,36133
1,6,30225
2,6 3,21139
3,11 6,12769
4,1,9766
5,11 6 3,9083
6,1 3,6002
7,11,5374
8,1 6,5046
9,4,4290


In [117]:
# Top-1024+ 4*128 SK with sequential search
# Total memory 9024 bytes :Top-1024 with size 9024 bytes.
# Execution time:358.27251648902893 seconds.


import sys
import time
import spookyhash
import mmh3
import hyperloglog
import operator
import unittest
from numpy import random

# ==========================data structure==========================
class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'count: {}'.format(self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self,count=1):
        super().__init__(count)
        self.distinct = hyperloglog.HyperLogLog(0.01)
        self.maxID=''
    def __str__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))
    def __repr__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
    def __repr__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
# ==========================data structure==========================

# ========================== Update Sketch==========================
def UpdateSk(element,width,depth):
    #　width+=1
    e_max=get_emax()
    # print("first read e_max in UpdateSK, id(e_max):{}".format(id(e_max)))
    hash1=spookyhash.hash32(bytes((element.ID),encoding='utf-8'))
        # input of spooky hash: binary, encoding is parameter of bytes()
        # output of spooky hash: unsigned- 32 bit int
    hash2=mmh3.hash(str(hash1), signed=False)
        # input of mmh3 is str, output unsigned- 32 bit int
    row=hash1 % depth
        # index of row in SK[row]
    ID=hash2 % ((width*3)//2)
        # hash-value(ID) of e
        # range: 2^k *3/2= 3*(2^k-1)
    match=False
        # match= True if e is in Sk[row]
    index=0
    item=Tail(ID,element.count)
    
    # print('UpdateSk:{}->ID {}'.format(element,ID))
    # print("hash1: {}".format(hash1))
    # print("hash2: {}".format(hash2))
    # print("match row: {}".format(row))
    # print("SK[{}]:{}".format(row,Sk[row]))
    # print("len(Sk[{}]:{})".format(row,len(Sk[row])))
    
    Sk[row][0].add_count(item.count)
        # total count+=count
    if len(Sk[row])==1:
        # Head only, append e directly
        Sk[row].append(item)
        Sk[row][0].maxID=element.ID
        match=True
        index=1
            # e is in Sk[row][1]
        # print('Sk[{}] when len(Sk[{}])==1:\n\t{}'.format(row,row,Sk[row]))
        # print("After append:\n{}".format(Sk[row]))
        # print("length of Sk[row]: {}".format(len(Sk[row])))
    elif len(Sk[row])<width:
        put_in=random.choice([0,1],size=1,p=[len(Sk[row])/width,1-len(Sk[row])/width])[0]
            # send into SK[row] with prob. 1-len(SK[row])/width
        for i in range(1,len(Sk[row])):
            # checks whether e is in SK[row]
            if ID == Sk[row][i].ID:
                match=True
                Sk[row][i].add_count(item.count)
                index=i
                '''
                print('{} matches in Sk[{}][{}]:{}'.format(item.ID,row,index,Sk[row][index].ID))
                print('Sk[{}]:\n\t{}'.format(row,Sk[row]))
                '''

                break
            else:
                match=False
        if not match:
            if put_in==1:
                # send e into SK[row]
                # print('put_in={}, send e into SK[{}]'.format(put_in,row))
                Sk[row].append(item)
                match=True
                index=len(Sk[row])-1
                # print('Sk[{}]:\n\t{}\n'.format(row,Sk[row]))
            else:
                Sk[row][0].distinct.add(ID)
                ''' 
                print('put_in={}, send e into other,update distinct'.format(put_in,len(Sk[row][0].distinct)))
                print('Sk[{}]:\n\t {}'.format(row,Sk[row]))
                '''

    elif len(Sk[row])==width:
        # len(Sk[row])=width, put e into other
        Sk[row][0].distinct.add(ID)
    # print("match:{}".format(match))
   
    # 這段會影響e_max
    if match:
        # print("e_max in UpdateSK line 114, id(e_max):{}".format(id(e_max)))
        if Sk[row][index].count>e_max.count:
            e_max.ID=element.ID
            e_max.count=Sk[row][index].count
        # print('line 115 e_max: e_max={}'.format(e_max))
            
    else:
        # avg count of other > e_max.count
        avg=(Sk[row][0].count-sum(Sk[row][i].count for i in range(len(Sk[row]))))/len(Sk[row][0].distinct)
        if avg >e_max.count:
            e_max=Tail(element.ID,avg)
        # print('line 122 e_max: e_max={}'.format(e_max))
    # print('line 123 e_max:{}'.format(get_emax()))
    # print('')
# ========================== Update Sketch==========================

# ========================== BringBack=========================
def BringBack(e_min,e_max):
    # print("Top before Bringback:\n\t{}".format(Top))
    e_max=get_emax()
    # print('e_max at first Bringback:{},id(e_max):{}'.format(e_max,id(e_max)))
    temp=Tail(e_min.ID,e_min.count)
    Top[-1].ID=e_max.ID
    Top[-1].count=e_max.count
    Top.sort(key=operator.attrgetter('count'),reverse=True)
    # print('Top after BringBack:\n\t{}'.format(Top))
    DeleteSk(e_max)
        # e_max in Sk[row]
    e_max.ID=''
    e_max.count=0
    # print("e_max after delete:{},id(e_max):{}".format(e_max,id(e_max)))
    UpdateSk(temp,width,depth)
    
    # print("Sk[] after Update {}:\n\t{}".format(e_min,Sk))

# ========================== BringBack=========================
# ==========================DeleteSk=========================
def DeleteSk(element):
    # 刪除e_max，找新的e_max
    # print('Delete start======================================')
    hash1=spookyhash.hash32(bytes((element.ID),encoding='utf-8'))
    hash2=mmh3.hash(str(hash1), signed=False)
    ID=hash2 % ((width*3)//2)
    row=hash1 % depth
    index=0
    for i in range(1,len(Sk[row])):
        if Sk[row][i].ID==ID:
            # print("found {} at SK[{}][{}], ID={}".format(element.ID,row,i,Sk[row][i].ID))
            index=i
            # print("pop element:{}".format(Sk[row][index]))
            Sk[row].pop(index)
            break
    # print('Delete Sk for find max of Sk[i] len(Sk):{}'.format(len(Sk)))
    Sk[row][0].count-=element.count
    '''
    print("index:{}".format(index))
    print('After pop:')
    print("\tSk[{}]={}".format(row,Sk[row]))
    print("Top:\n\t{}".format(Top))
    print('Delete Sketch over======================================')
    '''
# ==========================DeleteSk=========================
# ========================== Util Functions==========================

def get_emax():
    return e_max
# ========================== Util Functions==========================

start=time.time()
filename='kosarak.dat'
filepath="..\\dataset\\"
src_data=os.path.join(filepath,filename)
depth=4
width=128
size=1024

Top=[]
Sk=[[Head(0)] for j in range(depth)]
# item_count=1000

e_max=Tail('',0)

with open(src_data,'r') as file:
    while True:
        element=file.readline().strip('\n')
        # print("read element: {}".format(element))
        if not element:
            break
        else:
            # Update_Top(Tail(element,1))
            # item_count-=1
            if len(Top)==0:
                Top.append(Tail(element,1))
            else:
                found=False
                for i in range(len(Top)):
                    if Top[i].ID==element:
                        found=True
                        Top[i].add_count(1)
                        break
                if not found:
                    if len(Top)<size:
                        Top.append(Tail(element,1))
                    else:
                        # print("e_max in main, id(e_max):{}".format(id(e_max)))
                        UpdateSk(Tail(element,1),width,depth)
            Top.sort(key=operator.attrgetter('count'),reverse=True)
            # print("Top:{}".format(Top))
            if e_max.count>Top[-1].count:
                BringBack(Top[-1],e_max)
                # bring back
                # 要去掉SK[row]中的e_max.ID及count的node
                # print('Top after BringBack: \n\t{}'.format(Top))

end=time.time()
print(Top[:20],len(Top))
print("Total memory {1} bytes :Top-{3}: {0} bytes, Sketch: {2} bytes."
      .format(sys.getsizeof(Top),sys.getsizeof(Top)+sys.getsizeof(Sk),sys.getsizeof(Sk),size))
print("Execution time:{} seconds.".format(str(end-start)))

#　conver Top into df    
templi=[]
for i in Top:
    templi.append([i.ID,i.count])

df=pd.DataFrame(templi,columns=['ID', 'Count'])
df.to_csv("..\\result\\kosarak\\My_kosarak_o_1024.csv",index=False)
df.head(20)


SyntaxError: invalid syntax (<ipython-input-117-bf4bed832023>, line 2)

In [112]:
# My Result compare
import pandas as pd
import os
path='..\\result\\kosarak'
groundtruth='kosarak_ground_truth.csv'
built_in='My_kosarak_replace_last.csv'
origin='My_kosarak_o_1024.csv'

grtruth=pd.read_csv(os.path.join(path,groundtruth))
My_result1=pd.read_csv(os.path.join(path,origin))

gli = grtruth.values.tolist()
li1= My_result1.values.tolist()

tp=0
fp=0
find=0
err=[]

for item in li1:
    for element in gli[:512]:
        if item[0]==element[0]:
            find+=1
            if item[1]==element[1]:
                tp+=1
            else:
                # err.append([item[0],element[1]-item[1]])
                fp+=1
print("Find:{},TP:{},FP:{}".format(find,tp,fp))
# print(err)

Find:42,TP:0,FP:42


In [1]:
# top-512+ 4*128 SK with index search
# Total memory 4368 bytes :Top-512: 4272 bytes, Sketch: 96 bytes.
# Execution time:321.4541349411011 seconds.

import sys
import os
import time
import spookyhash
import mmh3
import hyperloglog
import operator
import unittest
import pandas as pd
from numpy import random

# ==========================data structure==========================
class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'count: {}'.format(self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self,count=1):
        super().__init__(count)
        self.distinct = hyperloglog.HyperLogLog(0.01)
        self.maxID=''
    def __str__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))
    def __repr__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
    def __repr__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
# ==========================data structure==========================

# ========================== Update Sketch==========================
def UpdateSk(element,width,depth):
    e_max=get_emax()
    hash1=spookyhash.hash32(bytes((element.ID),encoding='utf-8'))
        # input of spooky hash: binary, encoding is parameter of bytes()
        # output of spooky hash: unsigned- 32 bit int
    hash2=mmh3.hash(str(hash1), signed=False)
        # input of mmh3 is str, output unsigned- 32 bit int
    row=hash1 % depth
        # index of row in SK[row]
    ID=hash2 % ((width*3)//2)
        # hash-value(ID) of e
        # range: 2^k *3/2= 3*(2^k-1)
    match=False
        # match= True if e is in Sk[row]
    index=0
    avg=0
    item=Tail(ID, element.count)
        # ID in Sketch is hash value
    print("\t{} -> {},send to Sk[{}]".format(element,item,row))
    # print("id(Sk[{}]):{}".format(row,id(Sk[row])))
    if len(Sk[row])==0:
        # Sk[row] is empty,append e directly
        Sk[row].append(item)
        # Sk[row][0].maxID=element.ID
        match=True
        index=0
    else:
        # Sk[row] is not empty
        index=find(item,Sk[row])
        print("index={}".format(index))
        if index >0:
            # matches in Sk[row]
            Sk[row][index].count+=item.count
            match=True
        else:
            # doesn't match in Sk[row]
            if len(Sk[row])<width:
                Sk[row].append(item)
                match=True
                index=len(Sk[row])-1
            else:
                # Sk[row] is full,
                count_sum=sum(Sk[row][i].count for i in range(len(Sk[row])))
                # print("count_sum={}".format(count_sum))
                # print("total count={}".format(Sk[row][0].count))
                if Sk_head[row].count==count_sum:
                    # no element in other
                    # print("no element in Other, send to distinct")
                    Sk_head[row].distinct.add(element.ID)
                    match=False
                elif Sk_head[row].count>count_sum:
                    avg=(Sk_head[row].count-count_sum)//len(Sk_head[row].distinct)
                    if avg>Sk[row][-1].count:
                        # print("avg>last one, update Sk[{}]".format(row))
                        Sk[row][-1].ID=item.ID
                        Sk[row][-1].count=avg
                        match=True
                        index=-1
                    else:
                        Sk_head[row].distinct.add(element.ID)
                        # print("avg< last one,send to distinct")
                        match=False
    Sk_head[row].count=Sk_head[row].count+item.count
        # total count+=count
    
    # update e_max in Sketch
    if match:
        # e.count in Sk[row]> e_max.count
        if Sk[row][index].count>e_max.count:
            e_max.ID=element.ID
            e_max.count=Sk[row][index].count
    else:
        # avg(e.count) in other > e_max.count
        if avg >e_max.count:
            e_max=Tail(element.ID,avg)
    Sk[row].sort(key=operator.attrgetter('count'),reverse=True)
    # print("after sorting id(Sk[{}]):{}".format(row,id(Sk[row])))

    for i in range(len(Sk)):
        print("Sk[{}]:{},{}".format(i,Sk_head[i],Sk[i]))
    print("e_max:{}\n".format(e_max))

# ========================== Update Sketch==========================
# ========================== BringBack=========================
def BringBack(e_min,e_max):
    # print("Top before Bringback:\n\t{}".format(Top))
    e_max=get_emax()
    # print('e_max at first Bringback:{},id(e_max):{}'.format(e_max,id(e_max)))
    temp=Tail(e_min.ID,e_min.count)
    Top[-1].ID=e_max.ID
    Top[-1].count=e_max.count
    Top.sort(key=operator.attrgetter('count'),reverse=True)
    # print('Top after BringBack:\n\t{}'.format(Top))
    DeleteSk(e_max)
        # e_max in Sk[row]
    e_max.ID=''
    e_max.count=0
    # print("e_max after delete:{},id(e_max):{}".format(e_max,id(e_max)))
    UpdateSk(temp,width,depth)
    
    # print("Sk[] after Update {}:\n\t{}".format(e_min,Sk))

# ========================== BringBack=========================
# ==========================DeleteSk=========================
def DeleteSk(element):
    # 刪除e_max in Sk[row]，找新的e_max
    hash1=spookyhash.hash32(bytes((element.ID),encoding='utf-8'))
    hash2=mmh3.hash(str(hash1), signed=False)
    ID=hash2 % ((width*3)//2)
    row=hash1 % depth
    Sk[row][0].count-=element.count
    index=find(ID,Sk[row])
    if index<0:
        pass
    else:
        Sk[row].pop(index)
    '''
    print("index:{}".format(index))
    print('After pop:')
    print("\tSk[{}]={}".format(row,Sk[row]))
    print("Top:\n\t{}".format(Top))
    '''
# ==========================DeleteSk=========================
# ========================== Util Functions==========================
def get_emax():
    return e_max

def find(e,element_list):
    try:
        # type(element_list[i].ID):int
        # print("e:{}".format(e))
        # print("element_list[1:]: {}".format(element_list[1:]))
        index=[ele.ID for ele in element_list].index(e.ID)
    except:
        # print("raise error")
        index=-99
    return index

def sort_Sk(element_list):
    tail=element_list[1:]
    tail.sort(key=operator.attrgetter('count'),reverse=True)
    return [element_list[0]]+tail
# ========================== Util Functions==========================

start=time.time()
filename='kosarak.dat'
filepath="..\\dataset\\"
src_data=os.path.join(filepath,filename)
depth=4
width=8
size=8

Top=[]
Sk_head=[Head(0) for j in range(depth)]
Sk=[[] for i in range(depth)]
item_count=50
e_max=Tail('',0)

with open(src_data,'r') as file:
    while item_count:
        element=file.readline().strip('\n')
        if not element:
            break
        else:
            # print("read element: {}".format(element))
            item_count-=1
            item=Tail(element,1)
            if len(Top)==0:
                Top.append(item)
                #　print("len(Top)==0, Top append:{}".format(Top))
            else:
                index=find(item,Top)
                # print("index={}".format(index))
                if index<0:
                    if len(Top)<size:
                        Top.append(item)
                        # print("index={},Top after append: {},\nlen(Top):{}".format(index,Top,len(Top)))
                    else:
                        # print("send {} into Sk".format(element))
                        UpdateSk(item,width,depth)
                else:
                    # print("update Top[{}]:".format(index))
                    Top[index].count+=1
                    # print(Top)
            Top.sort(key=operator.attrgetter('count'),reverse=True)
            if e_max.count>Top[-1].count:
                BringBack(Top[-1],e_max)
                    # bring e_max back
                # print('Top after BringBack: \n\t{}'.format(Top))

end=time.time()
print("Top[20]:\n{},\nlen(Top):{}".format(Top[:20],len(Top)))
print("Total count:{}".format(sum(i.count for i in Top)+sum(j[0].count for j in Sk)))
print("Total memory {1} bytes :Top-{3}: {0} bytes, Sketch: {2} bytes."
      .format(sys.getsizeof(Top),sys.getsizeof(Top)+sys.getsizeof(Sk),sys.getsizeof(Sk),size))
print("Execution time:{} seconds.".format(str(end-start)))

#　conver Top into df    
templi=[]
for i in Top:
    templi.append([i.ID,i.count])

df=pd.DataFrame(templi,columns=['ID', 'Count'])
df.to_csv("..\\result\\kosarak\\My_kosarak_index_UpdateSK.csv",index=False)
df.head(20)




	[ID: 145 11 146, count: 1] -> [ID: 11, count: 1],send to Sk[2]
Sk[0]:[total count: 0, distinct: 0],[]
Sk[1]:[total count: 0, distinct: 0],[]
Sk[2]:[total count: 1, distinct: 0],[[ID: 11, count: 1]]
Sk[3]:[total count: 0, distinct: 0],[]
e_max:[ID: 145 11 146, count: 1]

	[ID: 147 3, count: 1] -> [ID: 2, count: 1],send to Sk[0]
Sk[0]:[total count: 1, distinct: 0],[[ID: 2, count: 1]]
Sk[1]:[total count: 0, distinct: 0],[]
Sk[2]:[total count: 1, distinct: 0],[[ID: 11, count: 1]]
Sk[3]:[total count: 0, distinct: 0],[]
e_max:[ID: 145 11 146, count: 1]

	[ID: 3 148, count: 1] -> [ID: 3, count: 1],send to Sk[2]
index=-99
Sk[0]:[total count: 1, distinct: 0],[[ID: 2, count: 1]]
Sk[1]:[total count: 0, distinct: 0],[]
Sk[2]:[total count: 2, distinct: 0],[[ID: 11, count: 1], [ID: 3, count: 1]]
Sk[3]:[total count: 0, distinct: 0],[]
e_max:[ID: 145 11 146, count: 1]

	[ID: 149, count: 1] -> [ID: 7, count: 1],send to Sk[0]
index=-99
Sk[0]:[total count: 2, distinct: 0],[[ID: 2, count: 1], [ID: 7, cou

ZeroDivisionError: integer division or modulo by zero

In [51]:
# My Result compare
import pandas as pd
import os
path='..\\result\\kosarak'
groundtruth='kosarak_ground_truth.csv'

final='My_kosarak_index_UpdateSK.csv'

grtruth=pd.read_csv(os.path.join(path,groundtruth))
My_result1=pd.read_csv(os.path.join(path,final))

gli = grtruth.values.tolist()
li1= My_result1.values.tolist()

tp=0
fp=0
find=0
err=[]

for item in li1:
    for element in gli[:512]:
        if item[0]==element[0]:
            find+=1
            if item[1]==element[1]:
                tp+=1
            else:
                # err.append([item[0],element[1]-item[1]])
                fp+=1
print("Find:{},TP:{},FP:{}".format(find,tp,fp))
# print(err)

Find:72,TP:72,FP:0


In [ ]:
https://ithelp.ithome.com.tw/articles/10161849